<p>
<img src="http://www.cerm.unifi.it/chianti/images/logo%20unifi_positivo.jpg" 
        alt="UniFI logo" style="float: left; width: 20%; height: 20%;">
<br>
<br>
<div align="right">
<b>Corso di Laurea Magistrale in Informatica</b><br>
<b>Curriculum Data Science</b><br>
</div>
</p>

<div align="right">
Matteo Marulli<br>
Matteo Ghera<br>
</div>

# Analisi dei dati del covid19 in Italia
Questa analisi dati è stata svolta prendendo i dati rilasciati dalla protezione civile sul covid19 in Italia.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from path import Path
import json
from IPython.display import YouTubeVideo

In [ ]:
PROJ_DIR = Path().getcwd().parent
DATA_DIR = PROJ_DIR / "data"
os.chdir (PROJ_DIR)

with open(DATA_DIR/'dpc-covid19-ita-province.json') as f:
         d= json.load(f)
json.dumps(d)
df_province =pd.DataFrame(d)

with open(DATA_DIR/'dpc-covid19-ita-regioni.json') as f:
         d= json.load(f)
json.dumps(d)
df_regioni =pd.DataFrame(d)

In [ ]:
df_province.head()

In [ ]:
df_regioni.head(20)

## Descrizione dei dataset
Il dataset **Regioni** è composto da 16 variaibili e molte variabili sono di tipo int64 e float 64.
Le variabili prese in analisi sono:
- *data*: è il timestamp indica quando è stato inserito il record nel database, deve essere convertito nel formato date
- *stato*: indica lo stato in è stato rilevato il caso (tipo qualitativo e non ordinale)
- *codice_regione*: codice istat per identificare una regione (tipo quantitativo e non ordinale)
- *denominazione_regione*: indica il nome della regione, (tipo qualitativo e non ordinale)
- *lat*: latidudine della regione (tipo quantitativo e ordinale)
- *long*: longitudine della regione (tipo quantitativo e ordinale)
- *ricoverati_con_sintomi*: numero di persone che sono state ricoverate nel giorno indicato da data (tipo quantitativo e ordianale)
- *terapia_intesiva*: numero di persone che sono state ricoverate/spostate in terapia intensiva nel giorno indicato da data (tipo quantitativo e ordianale)
- *totale_ospedalizzati*: numero di persone che sono state ricoverate in ospedale (somma tra quelli in tp.intensiva e ricoverati con sintomi) nel giorno indicato da data (tipo quantitativo e ordianale)
- *isolamento_domiciliare*: numero di persone che sono insolamento_domiciliare nel giorno indicato da data (tipo quantitativo e ordianale)
- *totale_attualmente_positivi*: numero di persone in isolamento e ricoverati in ospedale nel giorno indicato da data (tipo quantitativo e ordianale)
- *nuovi_attualmente_positivi*: differenza di persone positive al Covid19 tra il giorno attuale e il giorno passato (tipo quantitativo e ordianale)
- *dimessi_guariti*: persone guarite dal covid19 nella giornata indicata da data (tipo quantitativo e ordianale)
- *deceduti*: persone morte a causa del covid19 nella giornata indicata da data (tipo quantitativo e ordianale)
- *totale_casi*: totale casi positivi nella giornata indicata da data (tipo quantitativo e ordianale)
- *tamponi*: numerod i campioni fatti dalla regione nella giornata indicata da data (tipo quantitativo e ordianale)

Una riga in questo dataset corrisponde ad una fotografia della situazione per le regione italiane in un determinato giorno.

Il dataset **Province** è composto da 12 variaibili.
Le variabili prese in analisi sono:
- *data*: è il timestamp indica quando è stato inserito il record nel database, deve essere convertito nel formato date (tipo qualitativo e non ordinale)
- *stato*: indica lo stato, verrà cancellata nel preprocessing (tipo qualitativo e non ordinale)
- *codice_regione*: codice istat per identificare una regione (tipo quantitativo e non ordinale)
- *denominazione_regione*: indica il nome della regione (tipo qualitativo e non ordinale)
- *lat*: latidudine della città (tipo quantitativo e ordinale)
- *long*: longitudine della città (tipo quantitativo e ordinale)
- *codice_provincia*: codice istat per identificare una provincia (tipo quantitativo e non ordinale)
- *denominanzione_provincia*: indica il nome della provincia (tipo qualitativo e non ordinale)
- *sigla_provincia*: sigla della provincia usate per le targe auto (tipo qualitativo e non ordinale)
- *totale_casi*: persone infette da covid19 (tipo quantitativo e ordianale)
- *note_it*: note informative in italiano (tipo qualitativo e non ordinale)
- *note_en*: note informative in inglese (tipo qualitativo e non ordinale)
- *casi_testati*: numero di persone sottoposte al test per covid-19


Le colonne in comune nei due dataset sono:
- *data*
- *stato*
- *note_en*
- *note_it*
- *codice_regione*
- *totale_casi*
- *denominazione_regione*
- *lat*
- *long*

Una riga in questo dataset corrisponde ad una fotografia della situazione per le province italiane in un determinato giorno.

Le colonne candidate per eseguire la *join* sono: *data* e *denominazione_regione*

In [ ]:
set(df_regioni.columns).intersection(set(df_province.columns))

## Data preparation

In [ ]:
joinDF = pd.merge(df_province, df_regioni, on= ['data', 'codice_regione'])
joinDF

## Data cleaning

In questa sezioni si elimineranno le colonne che riteniamo non necessarie perchè forniscono informazioni ridondanti: *stato, codice_regione, note_en, note_it, codice_regione, codice_provincia* e *sigla_provincia*. Dopo che abbiamo eseguito la *join* sono nate delle colonne, indicate da \_x e da \_y, che abbiamo rimosso rimuovendo il suffisso finale. 

Successivamente abbiamo rinomato le colonne che avevano lo stesso nome ma valori diversi e abbiamo rimpiazzato i valori `NaN` della colonna *casi_testati* con la media dei valori nella colonna.

In [ ]:
joinDF.rename({"totale_casi_x":"totale_casi_provincia", "totale_casi_y":"totale_casi_regioni", "lat_x": "lat_provincia", "lat_y":"lat_regione","long_x":"long_provincia","long_y":"long_regione"}, axis=1, inplace=True)

In [ ]:
joinDF.columns = joinDF.columns.str.replace('_x','')
joinDF.drop(columns=["denominazione_regione_y"], inplace=True)
joinDF.columns = joinDF.columns.str.replace('_y','')
joinDF['data'] = pd.to_datetime(joinDF['data'], format='%Y-%m-%d') 
joinDF.drop(columns=['stato', 'codice_regione', 'note_en', 'note_it', 'codice_regione', 'codice_provincia','sigla_provincia'], axis=1, inplace = True)
joinDF['casi_testati'].fillna(joinDF['casi_testati'].mean(), inplace = True)
joinDF

## Data exploration

In questa fase cercheremo di rispondere a queste domande:

- Quali sono state le regioni con più ricoverati?*
- Quali sono state le regioni che hanno avuto più vittime? E quali sono quello con meno vittime?*
- Quali sono state le regioni con più malati?*
- Quanti tamponi sono stati fatti e chi ne ha fatti di più? *
- E’ stata superata la capienza nazionale dei posti letto?
- E’ stata superata la capienza regionale dei posti letto?
- Quanto è costata alla regione curare un malato covid in terapia intensiva?*
- Quanto è costato allo stato curare un malato covid in terapia intensiva?*
- Quanto soldi ha speso ogni regione per fare i tamponi per il covid?*
- Quanto soldi ha speso lo stato per fare i tamponi per il covid?*
- Quali sono state le regioni che hanno gestito meglio l’emergenza sanitaria?
- Quali sono state le regioni che hanno gestito meglio l’emergenza sanitaria?

Le domande precedenti possono essere eseguite anche per le province.

Risorse: <br>
https://it.businessinsider.com/terapia-intensive-coronavirus-quanto-costa-allo-stato-italiano-un-paziente-grave/ <br>
https://www.telemessina.it/2020/03/24/come-si-effettua-e-quanto-costa-un-tampone-per-covid-19/


In [ ]:
plt.style.use('seaborn-pastel')

In [ ]:
joinDF.info(memory_usage=True)

In [ ]:
joinDF.describe()

In [ ]:
plt.figure(figsize=(10,5), dpi = 150)
sns.heatmap(joinDF.corr(), annot=True, linewidths= 0.5 , cmap = 'coolwarm')

> *Quali sono state le regioni con più malati?*

In [ ]:
casiPositviPerRegioni = joinDF.pivot_table('totale_positivi', index=['data'], columns=['denominazione_regione'], aggfunc='max' )
casiPositviPerRegioni.plot(figsize=(20,14), marker='s', fontsize=22)
plt.title('Grafico dei casi positivi per regioni', size = 22)
plt.xlabel('Data',size = 22)
plt.ylabel('Numero di casi positivi', size = 22 )
plt.legend(loc=(1,0.54))

> *Quanti tamponi sono stati fatti e chi ne ha fatti di più?*

In [ ]:
tamponiPerRegioni = joinDF.pivot_table('tamponi', index=['data'], columns=['denominazione_regione'], aggfunc='max' )
tamponiPerRegioni.plot(figsize=(20,14), marker='s', fontsize=22)
plt.title('Grafico dei tamponi fatti per regioni', size = 22)
plt.xlabel('Data',size = 22)
plt.ylabel('Numero di tamponi fatti', size = 22 )
plt.legend(loc=(1,0.54))

>*Quanto soldi ha speso ogni regione per fare i tamponi per il covid?*

In [ ]:
numero_tamponi_regione=np.max(tamponiPerRegioni)
costo_tamponi_regione=numero_tamponi_regione*30
costo_tamponi_regione.plot(kind="bar", figsize=(12,6))
plt.title("Costo tamponi per regione", size=22)
plt.ylabel("Costo tampone", size=22)
plt.xlabel("Regione", size=22)
plt.xticks(size=22)
plt.yticks(size=22)

>*Quanto soldi ha speso lo stato per fare i tamponi per il covid?*

In [ ]:
np.sum(costo_tamponi_regione)

> *Quali sono state le regioni che hanno avuto più vittime? E quali sono quello con meno vittime?*

In [ ]:
decedutiPerRegioni = joinDF.pivot_table('deceduti', index=['data'], columns=['denominazione_regione'], aggfunc='max' )
decedutiPerRegioni.plot(figsize=(20,14), marker='s', fontsize=22)
plt.title('Grafico dei deceduti per regioni', size = 22)
plt.xlabel('Data',size = 22)
plt.ylabel('Numero di deceduti', size = 22 )
plt.legend(loc=(1,0.54))

>*Quali sono state le regioni con più ricoverati?*

In [ ]:
ospedalizzatiPerRegioni = joinDF.pivot_table(values = 'totale_ospedalizzati', index=['data'], columns=['denominazione_regione'], aggfunc='max' )
ospedalizzatiPerRegioni.plot(figsize=(20,14), marker='s', fontsize=22)
plt.title('Grafico delle persone ricoverate in ospedale per regioni', size = 22)
plt.xlabel('Data',size = 22)
plt.ylabel('Numero di persone ricoverate', size = 22 )
plt.legend(loc=(1,0.54))
plt.grid()

In [ ]:
#ospedalizzatiPerRegioni.boxplot(figsize=(20,14), fontsize=22)
plt.figure(figsize = (22,16))
sns.boxplot(data= ospedalizzatiPerRegioni)
plt.title('Grafico delle persone ricoverate in ospedale per regioni', size = 22)
plt.xlabel('Regioni',size = 22)
plt.xticks(rotation=90)
plt.ylabel('Numero di ricoverati', size = 22 )
plt.legend(loc=(1,0.54))
plt.grid()

>*Quanto è costata alla regione curare un malato covid in terapia intensiva?*

In [ ]:
terapiaIntsPerRegioni = joinDF.pivot_table(values = 'terapia_intensiva', index=['data'], columns=['denominazione_regione'], aggfunc='max' )
terapiaIntsPerRegioni.plot(figsize=(20,14), marker='s', fontsize=22)
plt.title("Grafico dell'andamento delle persone ricoverate in terapia intesivo per regioni", size = 22)
plt.xlabel('Data',size = 22)
plt.ylabel('Numero di persone ricoverate in tp. intensiva', size = 22 )
plt.legend(loc=(1,0.54))
plt.grid()

In [ ]:
terapiaIntsPerRegioni.boxplot(figsize=(20,14), fontsize=22)
plt.title('Boxplot delle persone ricoverate in terapia intensiva per regioni', size = 22)
plt.xlabel('Data',size = 22)
plt.xticks(rotation=90)
plt.ylabel('Numero di ricoverati', size = 22 )
plt.legend(loc=(1,0.54))

In [ ]:
costi_medi_terapia_intensiva=terapiaIntsPerRegioni.median()*1500*14
costi_medi_terapia_intensiva.plot(kind='barh', figsize=(20,14))
plt.title('Quanto hanno speso mediamente le regioni per il recovero dei malati in terapia intensiva?', size = 22)
plt.xlabel('Soldi spesi',size = 22)
plt.xticks(size=22)
plt.yticks(size=22)
plt.ylabel('Regioni', size = 22 )

>*Quanto è costato mediamente allo stato curare un malato covid in terapia intensiva?*

In [ ]:
np.sum(costi_medi_terapia_intensiva)

## Data prediction con il modello SIR
<img src="https://lacittanews.it/wp-content/uploads/2020/04/702635-thumb-full-720-160420merkel.jpg" 
        alt="UniFI logo" style="float: center; width: 40%; height: 36%;">
        



In [ ]:
YouTubeVideo('aBO9gvgiuVA')

In [ ]:
def SIR(y, t, n, beta, gamma):
    S, I, R = y
    dSdt = -(beta*S*I)/n
    dRdt = gamma*I
    dIdt = -dSdt -dRdt
    return dSdt, dIdt, dRdt

In [ ]:
from scipy.integrate import odeint

# Total population, N.
N = 10e6 #numero di abitanti in lombardia
# Initial number of infected and recovered individuals, I0 and R0.
I0, R0 = 176, 0 #si comincia con un infetto e zero guariti/morti
# Everyone else, S0, is susceptible to infection initially.
S0 = N - I0 - R0 # ci ricaviamo il numero di persone sane
# Contact rate, beta, and mean recovery rate, gamma, (in 1/days).
beta, gamma = 0.35, 1./10
# A grid of time points (in days)
t = np.linspace(0, 90, 91) #tempo in numero di giorni

y0 = S0, I0, R0
# Integrate the SIR equations over the time grid, t.
ret = odeint(SIR, y0, t, args=(N, beta, gamma))
S, I, R = ret.T

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(S, lw = 4, color = 'blue', linestyle ='--', label='S(t): individui sani')
plt.plot(I, lw = 4, color = 'red',linestyle =':', label='I(t): ammalati')
plt.plot(R, lw = 4, color = 'green',linestyle = '-.', label='R(t): persone guarite/decedute')
plt.title('Modello SIR per la Lombardia', size = 22)
plt.xlabel('Giorni', size = 22)
plt.ylabel('Numero di persone', size = 22)
plt.legend()
plt.grid()

In [ ]:
# Total population, N.
N = 3700000 #numero di abitanti in lombardia
# Initial number of infected and recovered individuals, I0 and R0.
I0, R0 = 1, 0 #si comincia con un infetto e zero guariti/morti
# Everyone else, S0, is susceptible to infection initially.
S0 = N - I0 - R0 # ci ricaviamo il numero di persone sane
# Contact rate, beta, and mean recovery rate, gamma, (in 1/days).
beta, gamma = 0.35, 1./10
# A grid of time points (in days)
t = np.linspace(0, 90, 91) #tempo in numero di giorni

y0 = S0, I0, R0
# Integrate the SIR equations over the time grid, t.
ret = odeint(SIR, y0, t, args=(N, beta, gamma))
S, I, R = ret.T

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(S, lw = 4, linestyle ='--', label='S(t): individui sani')
plt.plot(I, lw = 4, linestyle =':', label='I(t): ammalati')
plt.plot(R, lw = 4, linestyle = '-.', label='R(t): persone guarite/decedute')
plt.title('Modello SIR per la Toscana', size = 22)
plt.xlabel('Giorni', size = 22)
plt.ylabel('Numero di persone', size = 22)
plt.legend()
plt.grid()

In [ ]:
np.linspace(0, 90, 91)

# Esperiments

In [ ]:
import seaborn as sns
titanic = sns.load_dataset('titanic')
titanic.head()

In [ ]:
titanic[titanic['sex']=='male']['survived'].mean()

In [ ]:
titanic.groupby('sex')[['survived']].mean()

In [ ]:
titanic.pivot_table('survived', index = 'sex', aggfunc='sum')

In [ ]:
titanic.groupby(['sex', 'class'])['survived'].aggregate('mean').unstack() # calcolo la media dei sopravissuti per ogni classe divisa tra maschi e femminie

In [ ]:
titanic.pivot_table('survived', index='sex', columns='class')

In [ ]:
age = pd.cut(titanic['age'], [0, 18, 80])
titanic.pivot_table('survived', ['sex', age], 'class') #calcola la media dei sopravissuti per ogni classe divisi per maschi e femmini di età 0, 18 e 18,80

In [ ]:
joinDF['terapia_intensiva']-joinDF['dimessi_guariti']-joinDF['deceduti']

In [ ]:
import seaborn as sns
titanic = sns.load_dataset('titanic')
titanic.head()

In [ ]:
titanic[titanic['sex']=='male']['survived'].mean()

In [ ]:
titanic.groupby('sex')[['survived']].mean()

In [ ]:
titanic.pivot_table('survived', index = 'sex', aggfunc='sum')

In [ ]:
titanic.groupby(['sex', 'class'])['survived'].aggregate('mean').unstack() # calcolo la media dei sopravissuti per ogni classe divisa tra maschi e femminie

In [ ]:
titanic.pivot_table('survived', index='sex', columns='class')

In [ ]:
age = pd.cut(titanic['age'], [0, 18, 80])
titanic.pivot_table('survived', ['sex', age], 'class') #calcola la media dei sopravissuti per ogni classe divisi per maschi e femmini di età 0, 18 e 18,80

In [ ]:
joinDF['terapia_intensiva']-joinDF['dimessi_guariti']-joinDF['deceduti']